In [ ]:
import pandas as pd
import pymysql

conn = None
cur = None

sql = ""

conn = pymysql.connect(host='172.21.27.128', user='aison', password='Rkdqnrlte1q', db='Offloading1', charset='utf8')
cur = conn.cursor()

sql = "LOAD DATA LOCAL INFILE '/home/aison/Offloading/Daily_Data/Daily_Data_1_1_'" + yesterday + "'.csv' INTO TABLE Offloading1.Daily_Offloading_1 character set utf8 FIELDS TERMINATED BY '\t'"
result = pd.read_sql_query(sql,conn)
info_lte = pd.DataFrame(result)



In [ ]:
import pandas as pd
import pymysql
import os
from datetime import datetime, timedelta

base_date = datetime.today() - timedelta(2)
yesterday = base_date.strftime("%Y%m%d")
check_date = base_date.strftime("%Y-%m-%d")
rm_date = datetime.today() - timedelta(3)
rm_date = rm_date.strftime("%Y%m%d")
file_path = '/home/aison/Offloading/Daily_Data/Daily_Data_1_1_' + yesterday + '.csv'

def data_preprocess():
    raw_data = pd.read_excel('/home/aison/Offloading/Daily_Data/Daily_Data_1_1.xlsx', sheet_name='Daily_Data_1_1')
    os.system('mv /home/aison/Offloading/Daily_Data/Daily_Data_1_1.xlsx /home/aison/Offloading/Daily_Data/Daily_Data_1_1_'+ yesterday + '.xlsx')
    process_df = raw_data.drop(labels=range(0,2), axis=0)
    process_df.to_csv(file_path, sep='\t', encoding='utf-8', header=False, index=False)

def insert_data():
    conn = None
    cur = None
    sql = ""

    conn = pymysql.connect(host='172.21.27.128', user='aison', password='Rkdqnrlte1q', db='Offloading1', charset='utf8')
    cur = conn.cursor()
    sql = "LOAD DATA LOCAL INFILE " + "'" + file_path + "'" + " INTO TABLE Offloading1.Daily_Offloading_1 character set utf8 FIELDS TERMINATED BY '\t'"

def data_check():
    conn = None
    cur = None
    sql = ""

    conn = pymysql.connect(host='172.21.27.128', user='aison', password='Rkdqnrlte1q', db='Offloading1', charset='utf8')
    cur = conn.cursor()
    sql = "SELECT base_date FROM Daily_Offloading_1 ORDER BY base_date DESC LIMIT 1;"
    result = pd.read_sql_query(sql,conn)
    offloading = pd.DataFrame(result)
    data_date = str(offloading['base_date'])
    
    if data_date is not check_date:
        insert_data()
    else:
        print(data_date + " data has already existed.")

def rm_data():
    os.system('rm -rf /home/aison/Offloading/Daily_Data/Daily_Data_1_1_'+ rm_date + '.xlsx')
    os.system('rm -rf /home/aison/Offloading/Daily_Data/Daily_Data_1_1_'+ rm_date + '.csv')

if __name__ == "__main__":
    data_preprocess()
    data_check()
    #rm_data()
